<a href="https://colab.research.google.com/github/HunterCLcode/t5fakereview/blob/main/t5FakeReviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal of this t5FakeReviews is creating Fake Reviews from just the product name or category or both.

Imports & Version Control

In [1]:
!pip install fastai==2.2.7 sentencepiece simpletransformers

In [2]:
from simpletransformers.t5 import T5Model, T5Args
from fastai.text.all import *
import pandas as pd
import sentencepiece
import os
import re

In [3]:
!nvidia-smi

Tue Mar 22 23:49:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Mount Drive

In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set Folder to use...
WORK_DIR = '/content/drive/MyDrive/CS 4V98 Shared Data'
os.makedirs(WORK_DIR, exist_ok = True) 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Download our data, which is from the amazon review database

In [5]:
%cd /content/drive/MyDrive/CS 4v98 Shared Data/Workspace

/content/drive/.shortcut-targets-by-id/1Uvya0pwhro_vsBtB37hM8n_qHMRFb5Tp/CS 4v98 Shared Data/Workspace


In [6]:
# Grab review csv
reviews = pd.read_csv(f"Office_processed_v3.csv", sep=",")
r = re.compile(r'<\|BOS\|>(.*)<\|SEP\|>(.*)<\|SEP\|>(\d)<\|SEP\|>(.*)<\|EOS\|>', re.MULTILINE | re.DOTALL)

# Append to dataframe
df = []
for review in reviews['reviewText'].values[:501000]:
    title, category, overall, reviewText = r.match(review).groups()
    category = ", ".join(category.split(',')[1:])

    # must have: prefix, then input text, then reviewText
    df.append(["Generate Review: ", category, reviewText])

# Apply names title for product name, category for categories, label for review number 1-5,
# and description as the review of the product
df = pd.DataFrame(df, columns=['prefix', 'input_text', 'target_text',])
df = df.sample(frac = 1.0, random_state = 43) # Shuffle all the rows 
df.head()

# Split into training and generation rows
train_df = df.iloc[:100000,:].astype(str)
generate_df = df.iloc[100000:100480,:].astype(str)

In [7]:
%cd /content/drive/MyDrive/CS 4V98 Temp


/content/drive/MyDrive/CS 4V98 Temp


Initialize our model

In [ ]:
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 256,
    "eval_batch_size": 128,
    "num_train_epochs": 1,
    "save_eval_checkpoints": False,
    "use_multiprocessing": False,
    "num_beams": None,
    "do_sample": True,
    "max_length": 50,
    "top_k": 50,
    "top_p": 0.95,
    "num_return_sequences": 1,
}

model = T5Model("t5","t5-base", args=model_args)
model.train_model(train_df, eval_data=generate_df)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/12500 [00:00<?, ?it/s]

In [ ]:
test_df = df.iloc[100480:101000,:].astype(str)
preds = model.predict(
    ["Generate Review: " + description for description in test_df["input_text"].tolist()]
)

questions = test_df["target_text"].tolist()

with open("t5_generated_reviews_sampling.txt", "w") as f:
    for i, desc in enumerate(test_df["input_text"].tolist()):
        print(desc)
        print(preds[i])
        print()

        f.write("Categories:\n")
        f.write(str(desc) + "\n\n")

        f.write("Real review:\n")
        f.write(questions[i] + "\n\n")

        f.write("Generated review:\n")
        f.write(str(preds[i]) + "\n")
        f.write("________________________________________________________________________________\n")
